In [62]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import os

IDIR = os.path.abspath('../data')

In [63]:
print('loading prior')
priors = pd.read_csv(os.path.join(IDIR,'order_products__prior.csv'), dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})

print('loading train')
train = pd.read_csv(os.path.join(IDIR,'order_products__train.csv'), dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})

print('loading orders')
orders = pd.read_csv(os.path.join(IDIR,'orders.csv'), dtype={
        'order_id': np.int32,
        'user_id': np.int32,
        'eval_set': 'category',
        'order_number': np.int16,
        'order_dow': np.int8,
        'order_hour_of_day': np.int8,
        'days_since_prior_order': np.float32})

print('loading products')
products = pd.read_csv(os.path.join(IDIR,'products.csv'), dtype={
        'product_id': np.uint16,
        'order_id': np.int32,
        'aisle_id': np.uint8,
        'department_id': np.uint8},
        usecols=['product_id', 'aisle_id', 'department_id'])

loading prior
loading train
loading orders
loading products


In [64]:
priors.head(5)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [65]:
orders.head(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [66]:
products.head(5)

,product_id,aisle_id,department_id
0,1,61,19
1,2,104,13
2,3,94,7
3,4,38,1
4,5,5,13


# each file : the number of records and columns

In [67]:
print('priors {}: {}'.format(priors.shape, ', '.join(priors.columns)))
print('orders {}: {}'.format(orders.shape, ', '.join(orders.columns)))
print('train {}: {}'.format(train.shape, ', '.join(train.columns)))

priors (32434489, 4): order_id, product_id, add_to_cart_order, reordered
orders (3421083, 7): order_id, user_id, eval_set, order_number, order_dow, order_hour_of_day, days_since_prior_order
train (1384617, 4): order_id, product_id, add_to_cart_order, reordered


# join orders and priors

In [68]:
print('add order info to priors')
orders.set_index('order_id', inplace=True, drop=False)
priors = priors.join(orders, on='order_id', rsuffix='_')
priors.drop('order_id_', inplace=True, axis=1)

add order info to priors


In [69]:
priors.head(5)

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,202279,prior,3,5,9,8.0
1,2,28985,2,1,202279,prior,3,5,9,8.0
2,2,9327,3,0,202279,prior,3,5,9,8.0
3,2,45918,4,1,202279,prior,3,5,9,8.0
4,2,30035,5,0,202279,prior,3,5,9,8.0


# make user data with priors

In [70]:
print('computing user f')
usr = pd.DataFrame()
usr['average_days_between_orders'] = orders.groupby('user_id')['days_since_prior_order'].mean().astype(np.float32)
usr['nb_orders'] = orders.groupby('user_id').size().astype(np.int16)

users = pd.DataFrame()
users['total_items'] = priors.groupby('user_id').size().astype(np.int16)
users['all_products'] = priors.groupby('user_id')['product_id'].apply(set)
users['total_distinct_items'] = (users.all_products.map(len)).astype(np.int16)

users = users.join(usr)
del usr

computing user f


In [71]:
users.head(5)

,total_items,all_products,total_distinct_items,average_days_between_orders,nb_orders
user_id,,,,,
1,59,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",18,19.000000,11
2,195,"{45066, 2573, 18961, 23, 32792, 1559, 22559, 1...",102,16.285715,15
3,88,"{17668, 44683, 48523, 21903, 14992, 21137, 324...",33,12.000000,13
4,18,"{21573, 42329, 17769, 35469, 37646, 1200, 1905...",17,17.000000,6
5,37,"{11777, 40706, 28289, 48775, 20754, 6808, 1398...",23,11.500000,5


In [30]:
# new = users[0:10]
# new

,total_items,all_products,total_distinct_items,average_days_between_orders,nb_orders
user_id,,,,,
1,59,"{17122, 196, 26405, 46149, 14084, 13032, 26088...",18,19.000000,11
2,195,"{45066, 2573, 18961, 23, 32792, 1559, 22559, 1...",102,16.285715,15
3,88,"{17668, 44683, 48523, 21903, 14992, 21137, 324...",33,12.000000,13
4,18,"{21573, 42329, 17769, 35469, 37646, 1200, 1905...",17,17.000000,6
5,37,"{11777, 40706, 28289, 48775, 20754, 6808, 1398...",23,11.500000,5
6,14,"{40992, 27521, 20323, 48679, 8424, 45007, 2190...",12,13.333333,4
7,206,"{11520, 35333, 519, 10504, 47623, 45066, 13198...",68,10.450000,21
8,49,"{11136, 8193, 17794, 26882, 39812, 24838, 651,...",36,23.333334,4
9,76,"{8834, 38277, 33787, 5002, 11790, 38159, 7952,...",58,22.000000,4



tmp = []
for prod_id in products.product_id:
    containing_check = lambda x : x.issuperset({product_id})
    tmp.append(users.all_products.apply(containing_check).sum())

products['usernum'] = pd.Series(data = tmp, index = products.product_id)

In [51]:
product

user_id
1     {17122, 196, 26405, 46149, 14084, 13032, 26088...
2     {45066, 2573, 18961, 23, 32792, 1559, 22559, 1...
3     {17668, 44683, 48523, 21903, 14992, 21137, 324...
4     {21573, 42329, 17769, 35469, 37646, 1200, 1905...
5     {11777, 40706, 28289, 48775, 20754, 6808, 1398...
6     {40992, 27521, 20323, 48679, 8424, 45007, 2190...
7     {11520, 35333, 519, 10504, 47623, 45066, 13198...
8     {11136, 8193, 17794, 26882, 39812, 24838, 651,...
9     {8834, 38277, 33787, 5002, 11790, 38159, 7952,...
10    {36865, 20995, 13829, 43014, 11782, 18441, 476...
Name: all_products, dtype: object

In [61]:
subset = new.iloc[6].all_products
subset
subset.issuperset({45066})
print ("d")

d


In [72]:
print('computing product f')
prods = pd.DataFrame()
prods['orders'] = priors.groupby(priors.product_id).size().astype(np.int32) 
#제품별 주문된 횟수
prods['reorders'] = priors['reordered'].groupby(priors.product_id).sum().astype(np.float32)
#제품별 재구매된 횟수
prods['reorder_rate'] = (prods.reorders / prods.orders).astype(np.float32)
products = products.join(prods, on='product_id')
products.set_index('product_id', drop=False, inplace=True)

del prods

computing product f


In [73]:
tmp = []
for prod_id in products.product_id:
    containing_check = lambda x : x.issuperset({product_id})
    tmp.append(users.all_products.apply(containing_check).sum())

products['usernum'] = pd.Series(data = tmp, index = products.product_id)

In [ ]:
products.head(5)